In [10]:

from mods import readdata as rd
from mods import savedata as sd
from mods import extractdata as ed
from mods import gmap as gm
from mods import connectDB as connDB
from mods import date_mod as dm
from dotenv import load_dotenv
import os
import ast
import numpy as np
import pandas as pd

In [2]:
load_dotenv()

True

In [13]:
# 讀取raw data
path = "data/raw/pet_establishment.csv"
df = rd.get_csv_data(path)
df

[✓] CSV檔案已取回



,ID,legaltype,legalname,legaladdress,busitem,animaltype,validnum,validdate,own_name,bos_name,rank_year,rank_code,rank_flag_1,rank_flag_2,rank_text,state_flag
0,15913,A010,貓悅良犬寵物精品店,臺北市大安區基隆路二段110號1樓及2樓,BC,狗、貓,特寵業字第V1120348號,2027/11/17 上午 12:00:00,宣家瑩,許玲玲,2024.0,B,Y,N,NaN,N
1,19029,A090,啟豐商行,桃園市桃園區文中一路25巷39號,AB,貓,特寵業繁字第C1140733號,2028/10/2 上午 12:00:00,黃韻蓁,袁子榕,NaN,NaN,NaN,NaN,NaN,N
2,19023,A210,偶然貓舍,臺南市永康區二王里41鄰南工街212巷3弄13號,ABC,貓,特寵業繁字第U1140846號,2028/10/7 上午 12:00:00,施寶美,劉品艾,NaN,NaN,NaN,NaN,NaN,N
3,17813,A060,沐光貓舍,臺中市南區工學北路357號十樓之3,ABC,貓,特寵業繁字第S1130016號,2027/3/14 上午 12:00:00,許小芸,許芸鳳、黃信捷,NaN,NaN,NaN,NaN,NaN,N
4,19041,A020,森光貓境貓舍,新北市板橋區雙十路2段46巷22弄5號1樓,ABC,貓,特寵業繁字第A1140069號,2028/10/19 上午 12:00:00,洪蔡佳芬,周宜潔,NaN,NaN,NaN,NaN,NaN,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5499,17,A040,澎澎犬舍,高雄市旗山區大福巷5號,AB,狗,特寵業繁字第W1080095號,2022/1/10 上午 12:00:00,張志雄,彭秀惠,2019.0,B,Y,N,NaN,B
5500,6,A040,東威犬舍,高雄市楠梓區藍昌路51號,ABC,狗,特寵業繁字第W1080128號,2022/3/14 上午 12:00:00,詹東旺,曾健倫,2019.0,B,Y,N,NaN,P
5501,4,A040,台灣犬研育場,高雄市鳥松區神農路335-1號,ABC,狗,特寵業繁字第W1070135號,2021/11/16 上午 12:00:00,周張月琴,周進欽,2019.0,B,Y,N,NaN,P
5502,3,A040,彩虹狗,高雄市林園區鳳林路一段169號,BC,NaN,特寵業字第Ｗ1070144號,2024/9/25 上午 12:00:00,吳慧珠,龔高益,2019.0,C,Y,N,NaN,P


In [14]:
# 執行ETL
# 去除不必要的欄位
df_filtered = df[["legalname", "legaladdress", "busitem", "animaltype", "state_flag"]]
df_filtered

,legalname,legaladdress,busitem,animaltype,state_flag
0,貓悅良犬寵物精品店,臺北市大安區基隆路二段110號1樓及2樓,BC,狗、貓,N
1,啟豐商行,桃園市桃園區文中一路25巷39號,AB,貓,N
2,偶然貓舍,臺南市永康區二王里41鄰南工街212巷3弄13號,ABC,貓,N
3,沐光貓舍,臺中市南區工學北路357號十樓之3,ABC,貓,N
4,森光貓境貓舍,新北市板橋區雙十路2段46巷22弄5號1樓,ABC,貓,N
...,...,...,...,...,...
5499,澎澎犬舍,高雄市旗山區大福巷5號,AB,狗,B
5500,東威犬舍,高雄市楠梓區藍昌路51號,ABC,狗,P
5501,台灣犬研育場,高雄市鳥松區神農路335-1號,ABC,狗,P
5502,彩虹狗,高雄市林園區鳳林路一段169號,BC,NaN,P


In [15]:
# busitem = C表示寄養，state_flag = N表示營業中
df_filtered = df_filtered[
    (~df_filtered["busitem"].str.contains("A", na=False))
    & (df_filtered["busitem"].str.contains("C", na=False))
    & (df_filtered["state_flag"] == "N")
]
df_filtered

,legalname,legaladdress,busitem,animaltype,state_flag
0,貓悅良犬寵物精品店,臺北市大安區基隆路二段110號1樓及2樓,BC,狗、貓,N
5,東森寵物雲股份有限公司花蓮玉里分公司,花蓮縣玉里鎮和平路96號,C,狗、貓,N
7,東森寵物雲股份有限公司左營左大分公司,高雄市左營區左營大路161、163號,BC,狗、貓,N
8,東森寵物雲股份有限公司楠梓興楠分公司,高雄市楠梓區興楠路60號,BC,狗、貓,N
9,樂沛寵物,高雄市楠梓區土庫一路10號,C,狗、貓,N
...,...,...,...,...,...
5242,東森寵物雲股份有限公司龜山巨蛋分公司,桃園市龜山區山德里萬壽路二段1379號,BC,狗、貓,N
5274,編緣貓企業社,臺中市南區南門路47號1樓,C,貓,N
5337,貓的旅坊,臺南市東區裕農路691號1-2樓,C,貓,N
5420,貓屋住宿用品店,雲林縣斗六市永安路77號,C,貓,N


In [16]:
# 找出屬於六都的機構
df_filtered["city"], df_filtered["district"] = zip(
    *df_filtered["legaladdress"].apply(ed.extract_city_district)
)
df_filtered = df_filtered[df_filtered["city"].notna()].reset_index(drop=True)
df_filtered

C:\Users\dinni\AppData\Local\Temp\ipykernel_22896\4290206223.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["city"], df_filtered["district"] = zip(
C:\Users\dinni\AppData\Local\Temp\ipykernel_22896\4290206223.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["city"], df_filtered["district"] = zip(


,legalname,legaladdress,busitem,animaltype,state_flag,city,district
0,貓悅良犬寵物精品店,臺北市大安區基隆路二段110號1樓及2樓,BC,狗、貓,N,臺北市,大安區
1,東森寵物雲股份有限公司左營左大分公司,高雄市左營區左營大路161、163號,BC,狗、貓,N,高雄市,左營區
2,東森寵物雲股份有限公司楠梓興楠分公司,高雄市楠梓區興楠路60號,BC,狗、貓,N,高雄市,楠梓區
3,樂沛寵物,高雄市楠梓區土庫一路10號,C,狗、貓,N,高雄市,楠梓區
4,艾妮寵物沙龍,臺北市萬華區德昌街121號1樓,C,狗,N,臺北市,萬華區
...,...,...,...,...,...,...,...
1264,小毛球寵物坊,臺中市南區臺中市南區國光路443號1樓,BC,狗、貓,N,臺中市,南區
1265,東森寵物雲股份有限公司龜山巨蛋分公司,桃園市龜山區山德里萬壽路二段1379號,BC,狗、貓,N,桃園市,龜山區
1266,編緣貓企業社,臺中市南區南門路47號1樓,C,貓,N,臺中市,南區
1267,貓的旅坊,臺南市東區裕農路691號1-2樓,C,貓,N,臺南市,東區


In [17]:
# 只留下名稱、地址、市和區
df_filtered = df_filtered[["legalname", "legaladdress", "city", "district"]]
df_filtered

,legalname,legaladdress,city,district
0,貓悅良犬寵物精品店,臺北市大安區基隆路二段110號1樓及2樓,臺北市,大安區
1,東森寵物雲股份有限公司左營左大分公司,高雄市左營區左營大路161、163號,高雄市,左營區
2,東森寵物雲股份有限公司楠梓興楠分公司,高雄市楠梓區興楠路60號,高雄市,楠梓區
3,樂沛寵物,高雄市楠梓區土庫一路10號,高雄市,楠梓區
4,艾妮寵物沙龍,臺北市萬華區德昌街121號1樓,臺北市,萬華區
...,...,...,...,...
1264,小毛球寵物坊,臺中市南區臺中市南區國光路443號1樓,臺中市,南區
1265,東森寵物雲股份有限公司龜山巨蛋分公司,桃園市龜山區山德里萬壽路二段1379號,桃園市,龜山區
1266,編緣貓企業社,臺中市南區南門路47號1樓,臺中市,南區
1267,貓的旅坊,臺南市東區裕農路691號1-2樓,臺南市,東區


In [18]:
# 修改欄位名稱
df_filtered = df_filtered.rename(
    columns={
        "legalname": "name",
        "legaladdress": "address",
    }
)
df_filtered

,name,address,city,district
0,貓悅良犬寵物精品店,臺北市大安區基隆路二段110號1樓及2樓,臺北市,大安區
1,東森寵物雲股份有限公司左營左大分公司,高雄市左營區左營大路161、163號,高雄市,左營區
2,東森寵物雲股份有限公司楠梓興楠分公司,高雄市楠梓區興楠路60號,高雄市,楠梓區
3,樂沛寵物,高雄市楠梓區土庫一路10號,高雄市,楠梓區
4,艾妮寵物沙龍,臺北市萬華區德昌街121號1樓,臺北市,萬華區
...,...,...,...,...
1264,小毛球寵物坊,臺中市南區臺中市南區國光路443號1樓,臺中市,南區
1265,東森寵物雲股份有限公司龜山巨蛋分公司,桃園市龜山區山德里萬壽路二段1379號,桃園市,龜山區
1266,編緣貓企業社,臺中市南區南門路47號1樓,臺中市,南區
1267,貓的旅坊,臺南市東區裕農路691號1-2樓,臺南市,東區


In [19]:
# 透過google api並傳送名稱與地址取得place_id
API_KEY = os.getenv("GOOGLE_MAP_KEY_CHGWYELLOW")
result = []

# enumerate會自動將被iterate的物件附上index
for i, (idx, row) in enumerate(df_filtered.iterrows()):
    query = f"{row['name']} {row['address']}"

    result.append(gm.get_place_id(API_KEY, query))
df_filtered["place_id"] = np.nan
df_filtered.loc[:, "place_id"] = result

df_filtered

C:\Users\dinni\AppData\Local\Temp\ipykernel_22896\3790517035.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['ChIJF5gC5M2rQjQRNVDYYkI6w38', 'ChIJt72w85gFbjQRvtDmqmEmqXA', 'ChIJI4yoCi0QbjQRaa3rLmwX9_E', 'ChIJd32-tW4PbjQRc_RTEEXslr8', 'ChIJB_fXrJggaDQRe8CohwASiLk', 'ChIJ5Y3o1WsfaDQRhfp-uecJY7M', None, 'ChIJ7UnTJv4VaTQR-w8teThNiVk', 'ChIJQSb-nP6lQjQRp25gPneo5Uc', 'ChIJjZwMA5o9aTQRmY3_WnL4jaQ', 'ChIJJ9uDquapQjQRhyjrOg-uwew', 'ChIJcXJcgHSpQjQRNDlpI-2FZdI', 'ChIJi_njB3agQjQRbkQpnS_gTsU', 'ChIJY2gsl0J1bjQRImL6pDP8rY0', 'ChIJo035d0GpQjQRxOnY9hF0O4k', 'ChIJpfglnWcDaDQROqnauibQ4pE', 'ChIJGbnLzrmvQjQRuwIv5MbOrhQ', 'ChIJ13VK7b0BaDQRrN1sAt5dXoc', 'ChIJTUliRwCFbjQRK2KpSduhEO8', 'ChIJUxVIw-apQjQRdGzFO9XnFKw', 'ChIJWde5Zsvni2MRU7nv4bWTcgM', 'ChIJxy3cuuepQjQR0DPQ2NeCtMo', 'ChIJHcIvT6ypQjQRc9gESCt3ZSY', 'ChIJ_0MqQfN3bjQRMjpqe8pRsx8', 'ChIJ13ahBwWnQjQRaG9OBkFDSwE', 'ChIJ6fIImR8faDQRzCnb_YE-ouw', 'ChIJqYpX9X0XaTQRpRiSE6nwFbg', '

,name,address,city,district,place_id
0,貓悅良犬寵物精品店,臺北市大安區基隆路二段110號1樓及2樓,臺北市,大安區,ChIJF5gC5M2rQjQRNVDYYkI6w38
1,東森寵物雲股份有限公司左營左大分公司,高雄市左營區左營大路161、163號,高雄市,左營區,ChIJt72w85gFbjQRvtDmqmEmqXA
2,東森寵物雲股份有限公司楠梓興楠分公司,高雄市楠梓區興楠路60號,高雄市,楠梓區,ChIJI4yoCi0QbjQRaa3rLmwX9_E
3,樂沛寵物,高雄市楠梓區土庫一路10號,高雄市,楠梓區,ChIJd32-tW4PbjQRc_RTEEXslr8
4,艾妮寵物沙龍,臺北市萬華區德昌街121號1樓,臺北市,萬華區,ChIJB_fXrJggaDQRe8CohwASiLk
...,...,...,...,...,...
1264,小毛球寵物坊,臺中市南區臺中市南區國光路443號1樓,臺中市,南區,ChIJk4sZr_c9aTQRuqpS93LGEF8
1265,東森寵物雲股份有限公司龜山巨蛋分公司,桃園市龜山區山德里萬壽路二段1379號,桃園市,龜山區,ChIJres4rvYeaDQRg9QzSaBXUmo
1266,編緣貓企業社,臺中市南區南門路47號1樓,臺中市,南區,ChIJFyHszAY9aTQRDwDp7dVRn5o
1267,貓的旅坊,臺南市東區裕農路691號1-2樓,臺南市,東區,ChIJNeOGh7l2bjQRotcUscNuI2c


In [20]:
df_filtered = df_filtered.dropna(subset="place_id")

In [22]:
# 透過place_id找到詳細資料
result = []
for _, row in df_filtered.iterrows():
    result.append(gm.gmap_info(row["name"], API_KEY, row["place_id"]))
df_checked = pd.DataFrame(result)
df_checked

,name,place_id,business_status,address,phone,opening_hours,types,rating,rating_total,longitude,latitude,map_url,website,newest_review
0,貓悅良犬寵物坊,ChIJF5gC5M2rQjQRNVDYYkI6w38,OPERATIONAL,106台灣臺北市大安區基隆路二段110號1樓,0227326189,"[星期一: 13:30 – 22:30, 星期二: 13:30 – 22:30, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.2,39.0,121.556998,25.029194,https://maps.google.com/?cid=9206266120076021813,None,2025-09-19
1,東森寵物 左營左大店,ChIJt72w85gFbjQRvtDmqmEmqXA,OPERATIONAL,813台灣高雄市左營區左營大路161號,075818668,"[星期一: 10:00 – 21:00, 星期二: 10:00 – 21:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.5,198.0,120.289283,22.684462,https://maps.google.com/?cid=8118062004232966334,https://www.etipets.com/,2024-10-20
2,東森寵物 楠梓興楠店,ChIJI4yoCi0QbjQRaa3rLmwX9_E,OPERATIONAL,811台灣高雄市楠梓區興楠路60號,073542228,"[星期一: 10:00 – 22:00, 星期二: 10:00 – 22:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.5,633.0,120.338369,22.731786,https://maps.google.com/?cid=17435430235799268713,https://www.etipets.com/,2023-10-13
3,LaiDua寵物美容＆寵物住宿（安親、貓咪寄宿、高雄寵物住宿）,ChIJd32-tW4PbjQRc_RTEEXslr8,OPERATIONAL,811台灣高雄市楠梓區土庫一路10號,0971757031,"[星期一: 10:30 – 19:00, 星期二: 10:30 – 19:00, 星期三: ...","[establishment, point_of_interest]",4.9,57.0,120.332154,22.735754,https://maps.google.com/?cid=13805481489121670259,https://www.facebook.com/laiduapetspahotel,2023-11-05
4,Nicole Pet Salon,ChIJB_fXrJggaDQRe8CohwASiLk,OPERATIONAL,337台灣桃園市大園區大觀路59號,033851297,"[星期一: 07:30 – 21:00, 星期二: 07:30 – 21:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.4,13.0,121.194366,25.064971,https://maps.google.com/?cid=13368955287334600827,None,2023-03-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1248,小毛球寵物坊,ChIJk4sZr_c9aTQRuqpS93LGEF8,OPERATIONAL,402台灣臺中市南區國光路443號,0919399621,"[星期一: 13:00 – 21:00, 星期二: 13:00 – 21:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",5.0,6.0,120.679761,24.118867,https://maps.google.com/?cid=6850193230308485818,https://www.facebook.com/furballspets/,2025-03-07
1249,東森寵物 桃園巨蛋店,ChIJres4rvYeaDQRg9QzSaBXUmo,OPERATIONAL,333台灣桃園市龜山區萬壽路二段1379號,033490666,"[星期一: 10:00 – 22:00, 星期二: 10:00 – 22:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.4,1962.0,121.325233,24.992488,https://maps.google.com/?cid=7661282262045152387,http://www.104pet.com.tw/,2025-08-13
1250,良辰吉喵貓旅館(全預約制),ChIJFyHszAY9aTQRDwDp7dVRn5o,OPERATIONAL,402台灣臺中市南區南門路47號,0423210631,"[星期一: 10:30 – 19:00, 星期二: 10:30 – 19:00, 星期三: ...","[establishment, point_of_interest]",4.9,11.0,120.681542,24.122188,https://maps.google.com/?cid=11141713982399250447,https://www.facebook.com/thetimewithcats/,2019-02-04
1251,貓の旅坊 （預約制） 純貓咪旅館及低壓貓洗浴沙龍,ChIJNeOGh7l2bjQRotcUscNuI2c,OPERATIONAL,701台灣臺南市東區裕農路691號,0905966368,"[星期一: 10:00 – 19:30, 星期二: 10:00 – 19:30, 星期三: ...","[establishment, point_of_interest]",5.0,107.0,120.237445,22.988941,https://maps.google.com/?cid=7431905596860061602,https://www.facebook.com/%E8%B2%93%E3%81%AE%E6...,2025-08-20


In [23]:
df_checked = df_checked.dropna(subset="place_id")
df_checked

,name,place_id,business_status,address,phone,opening_hours,types,rating,rating_total,longitude,latitude,map_url,website,newest_review
0,貓悅良犬寵物坊,ChIJF5gC5M2rQjQRNVDYYkI6w38,OPERATIONAL,106台灣臺北市大安區基隆路二段110號1樓,0227326189,"[星期一: 13:30 – 22:30, 星期二: 13:30 – 22:30, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.2,39.0,121.556998,25.029194,https://maps.google.com/?cid=9206266120076021813,None,2025-09-19
1,東森寵物 左營左大店,ChIJt72w85gFbjQRvtDmqmEmqXA,OPERATIONAL,813台灣高雄市左營區左營大路161號,075818668,"[星期一: 10:00 – 21:00, 星期二: 10:00 – 21:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.5,198.0,120.289283,22.684462,https://maps.google.com/?cid=8118062004232966334,https://www.etipets.com/,2024-10-20
2,東森寵物 楠梓興楠店,ChIJI4yoCi0QbjQRaa3rLmwX9_E,OPERATIONAL,811台灣高雄市楠梓區興楠路60號,073542228,"[星期一: 10:00 – 22:00, 星期二: 10:00 – 22:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.5,633.0,120.338369,22.731786,https://maps.google.com/?cid=17435430235799268713,https://www.etipets.com/,2023-10-13
3,LaiDua寵物美容＆寵物住宿（安親、貓咪寄宿、高雄寵物住宿）,ChIJd32-tW4PbjQRc_RTEEXslr8,OPERATIONAL,811台灣高雄市楠梓區土庫一路10號,0971757031,"[星期一: 10:30 – 19:00, 星期二: 10:30 – 19:00, 星期三: ...","[establishment, point_of_interest]",4.9,57.0,120.332154,22.735754,https://maps.google.com/?cid=13805481489121670259,https://www.facebook.com/laiduapetspahotel,2023-11-05
4,Nicole Pet Salon,ChIJB_fXrJggaDQRe8CohwASiLk,OPERATIONAL,337台灣桃園市大園區大觀路59號,033851297,"[星期一: 07:30 – 21:00, 星期二: 07:30 – 21:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.4,13.0,121.194366,25.064971,https://maps.google.com/?cid=13368955287334600827,None,2023-03-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1248,小毛球寵物坊,ChIJk4sZr_c9aTQRuqpS93LGEF8,OPERATIONAL,402台灣臺中市南區國光路443號,0919399621,"[星期一: 13:00 – 21:00, 星期二: 13:00 – 21:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",5.0,6.0,120.679761,24.118867,https://maps.google.com/?cid=6850193230308485818,https://www.facebook.com/furballspets/,2025-03-07
1249,東森寵物 桃園巨蛋店,ChIJres4rvYeaDQRg9QzSaBXUmo,OPERATIONAL,333台灣桃園市龜山區萬壽路二段1379號,033490666,"[星期一: 10:00 – 22:00, 星期二: 10:00 – 22:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.4,1962.0,121.325233,24.992488,https://maps.google.com/?cid=7661282262045152387,http://www.104pet.com.tw/,2025-08-13
1250,良辰吉喵貓旅館(全預約制),ChIJFyHszAY9aTQRDwDp7dVRn5o,OPERATIONAL,402台灣臺中市南區南門路47號,0423210631,"[星期一: 10:30 – 19:00, 星期二: 10:30 – 19:00, 星期三: ...","[establishment, point_of_interest]",4.9,11.0,120.681542,24.122188,https://maps.google.com/?cid=11141713982399250447,https://www.facebook.com/thetimewithcats/,2019-02-04
1251,貓の旅坊 （預約制） 純貓咪旅館及低壓貓洗浴沙龍,ChIJNeOGh7l2bjQRotcUscNuI2c,OPERATIONAL,701台灣臺南市東區裕農路691號,0905966368,"[星期一: 10:00 – 19:30, 星期二: 10:00 – 19:30, 星期三: ...","[establishment, point_of_interest]",5.0,107.0,120.237445,22.988941,https://maps.google.com/?cid=7431905596860061602,https://www.facebook.com/%E8%B2%93%E3%81%AE%E6...,2025-08-20


In [24]:
df_merged = df_filtered.merge(
    df_checked,
    how="outer",
    left_on=df_filtered["place_id"],
    right_on=df_checked["place_id"],
    suffixes=["_filtered", "_checked"]
)
df_merged = df_merged.drop(columns=["place_id_filtered", "place_id_checked"])
df_merged = df_merged.drop_duplicates(subset="key_0")
df_merged

,key_0,name_filtered,address_filtered,city,district,name_checked,business_status,address_checked,phone,opening_hours,types,rating,rating_total,longitude,latitude,map_url,website,newest_review
0,ChIJ-1CF0w4daDQRRcdlKCG0yes,喵屋日和商行,新北市板橋區合宜路13號1樓,新北市,板橋區,喵屋喵wu,OPERATIONAL,220台灣新北市板橋區合宜一路27號,0922228719,"[星期一: 11:00 – 19:30, 星期二: 11:00 – 19:30, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.9,36.0,121.443503,24.996627,https://maps.google.com/?cid=16990309123736586053,https://www.instagram.com/miuwumiuwu2020?igsh=...,2025-08-29
1,ChIJ-2CGR413bjQRtSti57eyD0A,心花朵朵開實業社,臺南市東區小東路364號1樓,臺南市,東區,小花朵寵物美容旅館《推薦寵物美容店》台南寵物皮膚調理｜住宿安親｜寄宿服務 ｜沐浴SPA｜專業...,OPERATIONAL,701台灣臺南市東區小東路364號,0968985129,"[星期一: 休息, 星期二: 10:30 – 19:30, 星期三: 10:30 – 19:...","[establishment, pet_store, point_of_interest, ...",4.6,48.0,120.233784,22.998341,https://maps.google.com/?cid=4616104646008777653,https://www.facebook.com/ooxx9898new2.0,2024-01-18
2,ChIJ-4q1aW-sQjQRIvwTiOkA-ho,雅雅愛犬專業用品店,臺北市內湖區臺北市內湖區環山路一段63號1樓,臺北市,內湖區,雅雅愛犬專業美容,OPERATIONAL,114台灣臺北市內湖區環山路一段63號,0227986018,"[星期一: 休息, 星期二: 10:00 – 19:00, 星期三: 10:00 – 19:...","[establishment, point_of_interest]",4.8,41.0,121.568218,25.086302,https://maps.google.com/?cid=1943867192174181410,None,2023-05-31
3,ChIJ-5pqWYI9aTQRV4cgWYhIMJI,卡地亞犬坊,臺中市東區和平街259號1樓、地下1樓,臺中市,東區,卡地亞犬舍,OPERATIONAL,401台灣臺中市東區和平街259号號,0933402020,None,"[establishment, pet_store, point_of_interest, ...",3.7,3.0,120.689596,24.135867,https://maps.google.com/?cid=10533999278867646295,http://www.cartier-kennel.com/,2020-01-21
4,ChIJ-6-t6kEFbjQRvxbTFcaRkaw,貓阿舍寵物旅館,高雄市三民區嫩江街129巷4號1、2樓,高雄市,三民區,貓阿舍精品貓旅館,OPERATIONAL,807台灣高雄市三民區嫩江街129巷4號,0928409122,"[星期一: 13:00 – 19:00, 星期二: 13:00 – 19:00, 星期三: ...","[establishment, point_of_interest]",5.0,6.0,120.298294,22.644752,https://maps.google.com/?cid=12434880326102357695,None,2025-06-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3313,ChIJzebFfxevQjQRJKrPXAQBrz8,林水梨商行,新北市淡水區淡金路38巷33-20號1樓,新北市,淡水區,春松水果店紅樹林店/水果禮盒/現切水果/現打果汁/冰沙/竿蓁林輕軌站,OPERATIONAL,251台灣新北市淡水區淡金路38巷35弄5號,0922232427,"[星期一: 休息, 星期二: 11:00 – 21:00, 星期三: 11:00 – 21:...","[establishment, food, grocery_or_supermarket, ...",4.2,10.0,121.457958,25.162027,https://maps.google.com/?cid=4588887663562435108,https://line.me/R/ti/g/VaI3n_jdku,2024-11-07
3314,ChIJzwFWz5AZaDQR9h1DEVsCiL8,牠說寵物生活館,桃園市八德區桃園市八德區大和里金和路85號1樓,桃園市,八德區,牠說寵物生活館/美容 住宿 安親,OPERATIONAL,334台灣桃園市八德區金和路85號,033636320,"[星期一: 09:00 – 19:00, 星期二: 09:00 – 19:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.9,101.0,121.295934,24.954817,https://maps.google.com/?cid=13801283648231775734,https://www.facebook.com/WOUDLIKE/,2022-03-28
3315,ChIJzwrXlEpTXTQRBY0aHtNV7cU,橘舍寵物沙龍旅館,新北市汐止區秀峰路144號1樓,新北市,汐止區,AP寵物旅館x澡堂（寵物造型、寵物皮膚調理、寵物住宿、寵物零食、寵物保養品、寵物中途、寵物長照）,OPERATIONAL,221台灣新北市汐止區吉祥街11巷8號,0924120216,None,"[establishment, point_of_interest]",4.8,48.0,121.666803,25.079955,https://maps.google.com/?cid=14262149960183745797,https://linkbio.co/6060214ZXCa10,2024-06-05
3316,ChIJzxWfqJMEbjQRnbtlnXD8oQY,迷妳寵物店,高雄市新興區復興一路14之3號,高雄市,新興區,迷妳寵物店,OPERATIONAL,800台灣高雄市新興區復興一路14-3號,072225588,"[星期一: 12:00 – 22:00, 星期二: 12:00 – 22:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.2,31.0,120.308137,22.628589,https://maps.google.com/?cid=477940596085210013,https://www.facebook.com/%E8%BF%B7%E5%A6%B3%E5...,2024-10-09


In [25]:
df_merged = df_merged.dropna(subset="key_0")
df_merged

,key_0,name_filtered,address_filtered,city,district,name_checked,business_status,address_checked,phone,opening_hours,types,rating,rating_total,longitude,latitude,map_url,website,newest_review
0,ChIJ-1CF0w4daDQRRcdlKCG0yes,喵屋日和商行,新北市板橋區合宜路13號1樓,新北市,板橋區,喵屋喵wu,OPERATIONAL,220台灣新北市板橋區合宜一路27號,0922228719,"[星期一: 11:00 – 19:30, 星期二: 11:00 – 19:30, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.9,36.0,121.443503,24.996627,https://maps.google.com/?cid=16990309123736586053,https://www.instagram.com/miuwumiuwu2020?igsh=...,2025-08-29
1,ChIJ-2CGR413bjQRtSti57eyD0A,心花朵朵開實業社,臺南市東區小東路364號1樓,臺南市,東區,小花朵寵物美容旅館《推薦寵物美容店》台南寵物皮膚調理｜住宿安親｜寄宿服務 ｜沐浴SPA｜專業...,OPERATIONAL,701台灣臺南市東區小東路364號,0968985129,"[星期一: 休息, 星期二: 10:30 – 19:30, 星期三: 10:30 – 19:...","[establishment, pet_store, point_of_interest, ...",4.6,48.0,120.233784,22.998341,https://maps.google.com/?cid=4616104646008777653,https://www.facebook.com/ooxx9898new2.0,2024-01-18
2,ChIJ-4q1aW-sQjQRIvwTiOkA-ho,雅雅愛犬專業用品店,臺北市內湖區臺北市內湖區環山路一段63號1樓,臺北市,內湖區,雅雅愛犬專業美容,OPERATIONAL,114台灣臺北市內湖區環山路一段63號,0227986018,"[星期一: 休息, 星期二: 10:00 – 19:00, 星期三: 10:00 – 19:...","[establishment, point_of_interest]",4.8,41.0,121.568218,25.086302,https://maps.google.com/?cid=1943867192174181410,None,2023-05-31
3,ChIJ-5pqWYI9aTQRV4cgWYhIMJI,卡地亞犬坊,臺中市東區和平街259號1樓、地下1樓,臺中市,東區,卡地亞犬舍,OPERATIONAL,401台灣臺中市東區和平街259号號,0933402020,None,"[establishment, pet_store, point_of_interest, ...",3.7,3.0,120.689596,24.135867,https://maps.google.com/?cid=10533999278867646295,http://www.cartier-kennel.com/,2020-01-21
4,ChIJ-6-t6kEFbjQRvxbTFcaRkaw,貓阿舍寵物旅館,高雄市三民區嫩江街129巷4號1、2樓,高雄市,三民區,貓阿舍精品貓旅館,OPERATIONAL,807台灣高雄市三民區嫩江街129巷4號,0928409122,"[星期一: 13:00 – 19:00, 星期二: 13:00 – 19:00, 星期三: ...","[establishment, point_of_interest]",5.0,6.0,120.298294,22.644752,https://maps.google.com/?cid=12434880326102357695,None,2025-06-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3313,ChIJzebFfxevQjQRJKrPXAQBrz8,林水梨商行,新北市淡水區淡金路38巷33-20號1樓,新北市,淡水區,春松水果店紅樹林店/水果禮盒/現切水果/現打果汁/冰沙/竿蓁林輕軌站,OPERATIONAL,251台灣新北市淡水區淡金路38巷35弄5號,0922232427,"[星期一: 休息, 星期二: 11:00 – 21:00, 星期三: 11:00 – 21:...","[establishment, food, grocery_or_supermarket, ...",4.2,10.0,121.457958,25.162027,https://maps.google.com/?cid=4588887663562435108,https://line.me/R/ti/g/VaI3n_jdku,2024-11-07
3314,ChIJzwFWz5AZaDQR9h1DEVsCiL8,牠說寵物生活館,桃園市八德區桃園市八德區大和里金和路85號1樓,桃園市,八德區,牠說寵物生活館/美容 住宿 安親,OPERATIONAL,334台灣桃園市八德區金和路85號,033636320,"[星期一: 09:00 – 19:00, 星期二: 09:00 – 19:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.9,101.0,121.295934,24.954817,https://maps.google.com/?cid=13801283648231775734,https://www.facebook.com/WOUDLIKE/,2022-03-28
3315,ChIJzwrXlEpTXTQRBY0aHtNV7cU,橘舍寵物沙龍旅館,新北市汐止區秀峰路144號1樓,新北市,汐止區,AP寵物旅館x澡堂（寵物造型、寵物皮膚調理、寵物住宿、寵物零食、寵物保養品、寵物中途、寵物長照）,OPERATIONAL,221台灣新北市汐止區吉祥街11巷8號,0924120216,None,"[establishment, point_of_interest]",4.8,48.0,121.666803,25.079955,https://maps.google.com/?cid=14262149960183745797,https://linkbio.co/6060214ZXCa10,2024-06-05
3316,ChIJzxWfqJMEbjQRnbtlnXD8oQY,迷妳寵物店,高雄市新興區復興一路14之3號,高雄市,新興區,迷妳寵物店,OPERATIONAL,800台灣高雄市新興區復興一路14-3號,072225588,"[星期一: 12:00 – 22:00, 星期二: 12:00 – 22:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.2,31.0,120.308137,22.628589,https://maps.google.com/?cid=477940596085210013,https://www.facebook.com/%E8%BF%B7%E5%A6%B3%E5...,2024-10-09


In [26]:
df_merged = df_merged[df_merged["business_status"] == "OPERATIONAL"]
df_merged

,key_0,name_filtered,address_filtered,city,district,name_checked,business_status,address_checked,phone,opening_hours,types,rating,rating_total,longitude,latitude,map_url,website,newest_review
0,ChIJ-1CF0w4daDQRRcdlKCG0yes,喵屋日和商行,新北市板橋區合宜路13號1樓,新北市,板橋區,喵屋喵wu,OPERATIONAL,220台灣新北市板橋區合宜一路27號,0922228719,"[星期一: 11:00 – 19:30, 星期二: 11:00 – 19:30, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.9,36.0,121.443503,24.996627,https://maps.google.com/?cid=16990309123736586053,https://www.instagram.com/miuwumiuwu2020?igsh=...,2025-08-29
1,ChIJ-2CGR413bjQRtSti57eyD0A,心花朵朵開實業社,臺南市東區小東路364號1樓,臺南市,東區,小花朵寵物美容旅館《推薦寵物美容店》台南寵物皮膚調理｜住宿安親｜寄宿服務 ｜沐浴SPA｜專業...,OPERATIONAL,701台灣臺南市東區小東路364號,0968985129,"[星期一: 休息, 星期二: 10:30 – 19:30, 星期三: 10:30 – 19:...","[establishment, pet_store, point_of_interest, ...",4.6,48.0,120.233784,22.998341,https://maps.google.com/?cid=4616104646008777653,https://www.facebook.com/ooxx9898new2.0,2024-01-18
2,ChIJ-4q1aW-sQjQRIvwTiOkA-ho,雅雅愛犬專業用品店,臺北市內湖區臺北市內湖區環山路一段63號1樓,臺北市,內湖區,雅雅愛犬專業美容,OPERATIONAL,114台灣臺北市內湖區環山路一段63號,0227986018,"[星期一: 休息, 星期二: 10:00 – 19:00, 星期三: 10:00 – 19:...","[establishment, point_of_interest]",4.8,41.0,121.568218,25.086302,https://maps.google.com/?cid=1943867192174181410,None,2023-05-31
3,ChIJ-5pqWYI9aTQRV4cgWYhIMJI,卡地亞犬坊,臺中市東區和平街259號1樓、地下1樓,臺中市,東區,卡地亞犬舍,OPERATIONAL,401台灣臺中市東區和平街259号號,0933402020,None,"[establishment, pet_store, point_of_interest, ...",3.7,3.0,120.689596,24.135867,https://maps.google.com/?cid=10533999278867646295,http://www.cartier-kennel.com/,2020-01-21
4,ChIJ-6-t6kEFbjQRvxbTFcaRkaw,貓阿舍寵物旅館,高雄市三民區嫩江街129巷4號1、2樓,高雄市,三民區,貓阿舍精品貓旅館,OPERATIONAL,807台灣高雄市三民區嫩江街129巷4號,0928409122,"[星期一: 13:00 – 19:00, 星期二: 13:00 – 19:00, 星期三: ...","[establishment, point_of_interest]",5.0,6.0,120.298294,22.644752,https://maps.google.com/?cid=12434880326102357695,None,2025-06-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3313,ChIJzebFfxevQjQRJKrPXAQBrz8,林水梨商行,新北市淡水區淡金路38巷33-20號1樓,新北市,淡水區,春松水果店紅樹林店/水果禮盒/現切水果/現打果汁/冰沙/竿蓁林輕軌站,OPERATIONAL,251台灣新北市淡水區淡金路38巷35弄5號,0922232427,"[星期一: 休息, 星期二: 11:00 – 21:00, 星期三: 11:00 – 21:...","[establishment, food, grocery_or_supermarket, ...",4.2,10.0,121.457958,25.162027,https://maps.google.com/?cid=4588887663562435108,https://line.me/R/ti/g/VaI3n_jdku,2024-11-07
3314,ChIJzwFWz5AZaDQR9h1DEVsCiL8,牠說寵物生活館,桃園市八德區桃園市八德區大和里金和路85號1樓,桃園市,八德區,牠說寵物生活館/美容 住宿 安親,OPERATIONAL,334台灣桃園市八德區金和路85號,033636320,"[星期一: 09:00 – 19:00, 星期二: 09:00 – 19:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.9,101.0,121.295934,24.954817,https://maps.google.com/?cid=13801283648231775734,https://www.facebook.com/WOUDLIKE/,2022-03-28
3315,ChIJzwrXlEpTXTQRBY0aHtNV7cU,橘舍寵物沙龍旅館,新北市汐止區秀峰路144號1樓,新北市,汐止區,AP寵物旅館x澡堂（寵物造型、寵物皮膚調理、寵物住宿、寵物零食、寵物保養品、寵物中途、寵物長照）,OPERATIONAL,221台灣新北市汐止區吉祥街11巷8號,0924120216,None,"[establishment, point_of_interest]",4.8,48.0,121.666803,25.079955,https://maps.google.com/?cid=14262149960183745797,https://linkbio.co/6060214ZXCa10,2024-06-05
3316,ChIJzxWfqJMEbjQRnbtlnXD8oQY,迷妳寵物店,高雄市新興區復興一路14之3號,高雄市,新興區,迷妳寵物店,OPERATIONAL,800台灣高雄市新興區復興一路14-3號,072225588,"[星期一: 12:00 – 22:00, 星期二: 12:00 – 22:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.2,31.0,120.308137,22.628589,https://maps.google.com/?cid=477940596085210013,https://www.facebook.com/%E8%BF%B7%E5%A6%B3%E5...,2024-10-09


In [27]:
# 檢查是否有空值
df_merged.isna().any()

key_0               False
name_filtered       False
address_filtered    False
city                False
district            False
name_checked        False
business_status     False
address_checked     False
phone                True
opening_hours        True
types               False
rating               True
rating_total         True
longitude           False
latitude            False
map_url             False
website              True
newest_review        True
dtype: bool

In [28]:
revised_columns = [
    "key_0",
    "name_filtered",
    "name_checked",
    "address_filtered",
    "address_checked",
    "phone",
    "city",
    "district",
    "business_status",
    "opening_hours",
    "types",
    "rating",
    "rating_total",
    "longitude",
    "latitude",
    "map_url",
    "website",
    "newest_review",
]
df_merged = df_merged[revised_columns]
df_merged

,key_0,name_filtered,name_checked,address_filtered,address_checked,phone,city,district,business_status,opening_hours,types,rating,rating_total,longitude,latitude,map_url,website,newest_review
0,ChIJ-1CF0w4daDQRRcdlKCG0yes,喵屋日和商行,喵屋喵wu,新北市板橋區合宜路13號1樓,220台灣新北市板橋區合宜一路27號,0922228719,新北市,板橋區,OPERATIONAL,"[星期一: 11:00 – 19:30, 星期二: 11:00 – 19:30, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.9,36.0,121.443503,24.996627,https://maps.google.com/?cid=16990309123736586053,https://www.instagram.com/miuwumiuwu2020?igsh=...,2025-08-29
1,ChIJ-2CGR413bjQRtSti57eyD0A,心花朵朵開實業社,小花朵寵物美容旅館《推薦寵物美容店》台南寵物皮膚調理｜住宿安親｜寄宿服務 ｜沐浴SPA｜專業...,臺南市東區小東路364號1樓,701台灣臺南市東區小東路364號,0968985129,臺南市,東區,OPERATIONAL,"[星期一: 休息, 星期二: 10:30 – 19:30, 星期三: 10:30 – 19:...","[establishment, pet_store, point_of_interest, ...",4.6,48.0,120.233784,22.998341,https://maps.google.com/?cid=4616104646008777653,https://www.facebook.com/ooxx9898new2.0,2024-01-18
2,ChIJ-4q1aW-sQjQRIvwTiOkA-ho,雅雅愛犬專業用品店,雅雅愛犬專業美容,臺北市內湖區臺北市內湖區環山路一段63號1樓,114台灣臺北市內湖區環山路一段63號,0227986018,臺北市,內湖區,OPERATIONAL,"[星期一: 休息, 星期二: 10:00 – 19:00, 星期三: 10:00 – 19:...","[establishment, point_of_interest]",4.8,41.0,121.568218,25.086302,https://maps.google.com/?cid=1943867192174181410,None,2023-05-31
3,ChIJ-5pqWYI9aTQRV4cgWYhIMJI,卡地亞犬坊,卡地亞犬舍,臺中市東區和平街259號1樓、地下1樓,401台灣臺中市東區和平街259号號,0933402020,臺中市,東區,OPERATIONAL,None,"[establishment, pet_store, point_of_interest, ...",3.7,3.0,120.689596,24.135867,https://maps.google.com/?cid=10533999278867646295,http://www.cartier-kennel.com/,2020-01-21
4,ChIJ-6-t6kEFbjQRvxbTFcaRkaw,貓阿舍寵物旅館,貓阿舍精品貓旅館,高雄市三民區嫩江街129巷4號1、2樓,807台灣高雄市三民區嫩江街129巷4號,0928409122,高雄市,三民區,OPERATIONAL,"[星期一: 13:00 – 19:00, 星期二: 13:00 – 19:00, 星期三: ...","[establishment, point_of_interest]",5.0,6.0,120.298294,22.644752,https://maps.google.com/?cid=12434880326102357695,None,2025-06-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3313,ChIJzebFfxevQjQRJKrPXAQBrz8,林水梨商行,春松水果店紅樹林店/水果禮盒/現切水果/現打果汁/冰沙/竿蓁林輕軌站,新北市淡水區淡金路38巷33-20號1樓,251台灣新北市淡水區淡金路38巷35弄5號,0922232427,新北市,淡水區,OPERATIONAL,"[星期一: 休息, 星期二: 11:00 – 21:00, 星期三: 11:00 – 21:...","[establishment, food, grocery_or_supermarket, ...",4.2,10.0,121.457958,25.162027,https://maps.google.com/?cid=4588887663562435108,https://line.me/R/ti/g/VaI3n_jdku,2024-11-07
3314,ChIJzwFWz5AZaDQR9h1DEVsCiL8,牠說寵物生活館,牠說寵物生活館/美容 住宿 安親,桃園市八德區桃園市八德區大和里金和路85號1樓,334台灣桃園市八德區金和路85號,033636320,桃園市,八德區,OPERATIONAL,"[星期一: 09:00 – 19:00, 星期二: 09:00 – 19:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.9,101.0,121.295934,24.954817,https://maps.google.com/?cid=13801283648231775734,https://www.facebook.com/WOUDLIKE/,2022-03-28
3315,ChIJzwrXlEpTXTQRBY0aHtNV7cU,橘舍寵物沙龍旅館,AP寵物旅館x澡堂（寵物造型、寵物皮膚調理、寵物住宿、寵物零食、寵物保養品、寵物中途、寵物長照）,新北市汐止區秀峰路144號1樓,221台灣新北市汐止區吉祥街11巷8號,0924120216,新北市,汐止區,OPERATIONAL,None,"[establishment, point_of_interest]",4.8,48.0,121.666803,25.079955,https://maps.google.com/?cid=14262149960183745797,https://linkbio.co/6060214ZXCa10,2024-06-05
3316,ChIJzxWfqJMEbjQRnbtlnXD8oQY,迷妳寵物店,迷妳寵物店,高雄市新興區復興一路14之3號,800台灣高雄市新興區復興一路14-3號,072225588,高雄市,新興區,OPERATIONAL,"[星期一: 12:00 – 22:00, 星期二: 12:00 – 22:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.2,31.0,120.308137,22.628589,https://maps.google.com/?cid=477940596085210013,https://www.facebook.com/%E8%BF%B7%E5%A6%B3%E5...,2024-10-09


In [29]:
if df_merged["key_0"].duplicated().any():
    df_merged = df_merged.drop_duplicates(subset="key_0")
df_merged

,key_0,name_filtered,name_checked,address_filtered,address_checked,phone,city,district,business_status,opening_hours,types,rating,rating_total,longitude,latitude,map_url,website,newest_review
0,ChIJ-1CF0w4daDQRRcdlKCG0yes,喵屋日和商行,喵屋喵wu,新北市板橋區合宜路13號1樓,220台灣新北市板橋區合宜一路27號,0922228719,新北市,板橋區,OPERATIONAL,"[星期一: 11:00 – 19:30, 星期二: 11:00 – 19:30, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.9,36.0,121.443503,24.996627,https://maps.google.com/?cid=16990309123736586053,https://www.instagram.com/miuwumiuwu2020?igsh=...,2025-08-29
1,ChIJ-2CGR413bjQRtSti57eyD0A,心花朵朵開實業社,小花朵寵物美容旅館《推薦寵物美容店》台南寵物皮膚調理｜住宿安親｜寄宿服務 ｜沐浴SPA｜專業...,臺南市東區小東路364號1樓,701台灣臺南市東區小東路364號,0968985129,臺南市,東區,OPERATIONAL,"[星期一: 休息, 星期二: 10:30 – 19:30, 星期三: 10:30 – 19:...","[establishment, pet_store, point_of_interest, ...",4.6,48.0,120.233784,22.998341,https://maps.google.com/?cid=4616104646008777653,https://www.facebook.com/ooxx9898new2.0,2024-01-18
2,ChIJ-4q1aW-sQjQRIvwTiOkA-ho,雅雅愛犬專業用品店,雅雅愛犬專業美容,臺北市內湖區臺北市內湖區環山路一段63號1樓,114台灣臺北市內湖區環山路一段63號,0227986018,臺北市,內湖區,OPERATIONAL,"[星期一: 休息, 星期二: 10:00 – 19:00, 星期三: 10:00 – 19:...","[establishment, point_of_interest]",4.8,41.0,121.568218,25.086302,https://maps.google.com/?cid=1943867192174181410,None,2023-05-31
3,ChIJ-5pqWYI9aTQRV4cgWYhIMJI,卡地亞犬坊,卡地亞犬舍,臺中市東區和平街259號1樓、地下1樓,401台灣臺中市東區和平街259号號,0933402020,臺中市,東區,OPERATIONAL,None,"[establishment, pet_store, point_of_interest, ...",3.7,3.0,120.689596,24.135867,https://maps.google.com/?cid=10533999278867646295,http://www.cartier-kennel.com/,2020-01-21
4,ChIJ-6-t6kEFbjQRvxbTFcaRkaw,貓阿舍寵物旅館,貓阿舍精品貓旅館,高雄市三民區嫩江街129巷4號1、2樓,807台灣高雄市三民區嫩江街129巷4號,0928409122,高雄市,三民區,OPERATIONAL,"[星期一: 13:00 – 19:00, 星期二: 13:00 – 19:00, 星期三: ...","[establishment, point_of_interest]",5.0,6.0,120.298294,22.644752,https://maps.google.com/?cid=12434880326102357695,None,2025-06-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3313,ChIJzebFfxevQjQRJKrPXAQBrz8,林水梨商行,春松水果店紅樹林店/水果禮盒/現切水果/現打果汁/冰沙/竿蓁林輕軌站,新北市淡水區淡金路38巷33-20號1樓,251台灣新北市淡水區淡金路38巷35弄5號,0922232427,新北市,淡水區,OPERATIONAL,"[星期一: 休息, 星期二: 11:00 – 21:00, 星期三: 11:00 – 21:...","[establishment, food, grocery_or_supermarket, ...",4.2,10.0,121.457958,25.162027,https://maps.google.com/?cid=4588887663562435108,https://line.me/R/ti/g/VaI3n_jdku,2024-11-07
3314,ChIJzwFWz5AZaDQR9h1DEVsCiL8,牠說寵物生活館,牠說寵物生活館/美容 住宿 安親,桃園市八德區桃園市八德區大和里金和路85號1樓,334台灣桃園市八德區金和路85號,033636320,桃園市,八德區,OPERATIONAL,"[星期一: 09:00 – 19:00, 星期二: 09:00 – 19:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.9,101.0,121.295934,24.954817,https://maps.google.com/?cid=13801283648231775734,https://www.facebook.com/WOUDLIKE/,2022-03-28
3315,ChIJzwrXlEpTXTQRBY0aHtNV7cU,橘舍寵物沙龍旅館,AP寵物旅館x澡堂（寵物造型、寵物皮膚調理、寵物住宿、寵物零食、寵物保養品、寵物中途、寵物長照）,新北市汐止區秀峰路144號1樓,221台灣新北市汐止區吉祥街11巷8號,0924120216,新北市,汐止區,OPERATIONAL,None,"[establishment, point_of_interest]",4.8,48.0,121.666803,25.079955,https://maps.google.com/?cid=14262149960183745797,https://linkbio.co/6060214ZXCa10,2024-06-05
3316,ChIJzxWfqJMEbjQRnbtlnXD8oQY,迷妳寵物店,迷妳寵物店,高雄市新興區復興一路14之3號,800台灣高雄市新興區復興一路14-3號,072225588,高雄市,新興區,OPERATIONAL,"[星期一: 12:00 – 22:00, 星期二: 12:00 – 22:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.2,31.0,120.308137,22.628589,https://maps.google.com/?cid=477940596085210013,https://www.facebook.com/%E8%BF%B7%E5%A6%B3%E5...,2024-10-09


In [30]:
# 計算欄位空值補0
fillna_columns = ["opening_hours", "rating", "rating_total", "newest_review"]
df_merged[fillna_columns] = df_merged[fillna_columns].fillna(0)
df_merged.isna().any()

key_0               False
name_filtered       False
name_checked        False
address_filtered    False
address_checked     False
phone                True
city                False
district            False
business_status     False
opening_hours       False
types               False
rating              False
rating_total        False
longitude           False
latitude            False
map_url             False
website              True
newest_review       False
dtype: bool

In [31]:
# 連線資料庫
host = os.getenv("MYSQL_IP")
port = int(os.getenv("MYSQL_PORTT"))
user = os.getenv("MYSQL_USERNAME")
password = os.getenv("MYSQL_PASSWORD")
db = os.getenv("MYSQL_DB_NAME")
conn, cursor = connDB.connect_db(host, port, user, password, db)

✅ TJR103_1資料庫已成功連線


In [32]:
# 讀取location表格的資料
sql = """
select loc_id, city, district
from location;
"""
cursor.execute(sql)
loc = cursor.fetchall()

df_loc = pd.DataFrame(data=loc, columns=["loc_id", "city", "district"])
df_loc

,loc_id,city,district
0,NTP001,新北市,板橋區
1,NTP002,新北市,三重區
2,NTP003,新北市,中和區
3,NTP004,新北市,永和區
4,NTP005,新北市,新莊區
...,...,...,...
153,KSH034,高雄市,杉林區
154,KSH035,高雄市,內門區
155,KSH036,高雄市,茂林區
156,KSH037,高雄市,桃源區


In [33]:
# merge df_merged和df_loc
df_final = df_merged.merge(df_loc, left_on=["city", "district"], right_on=["city", "district"], how="left")
df_final

,key_0,name_filtered,name_checked,address_filtered,address_checked,phone,city,district,business_status,opening_hours,types,rating,rating_total,longitude,latitude,map_url,website,newest_review,loc_id
0,ChIJ-1CF0w4daDQRRcdlKCG0yes,喵屋日和商行,喵屋喵wu,新北市板橋區合宜路13號1樓,220台灣新北市板橋區合宜一路27號,0922228719,新北市,板橋區,OPERATIONAL,"[星期一: 11:00 – 19:30, 星期二: 11:00 – 19:30, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.9,36.0,121.443503,24.996627,https://maps.google.com/?cid=16990309123736586053,https://www.instagram.com/miuwumiuwu2020?igsh=...,2025-08-29,NTP001
1,ChIJ-2CGR413bjQRtSti57eyD0A,心花朵朵開實業社,小花朵寵物美容旅館《推薦寵物美容店》台南寵物皮膚調理｜住宿安親｜寄宿服務 ｜沐浴SPA｜專業...,臺南市東區小東路364號1樓,701台灣臺南市東區小東路364號,0968985129,臺南市,東區,OPERATIONAL,"[星期一: 休息, 星期二: 10:30 – 19:30, 星期三: 10:30 – 19:...","[establishment, pet_store, point_of_interest, ...",4.6,48.0,120.233784,22.998341,https://maps.google.com/?cid=4616104646008777653,https://www.facebook.com/ooxx9898new2.0,2024-01-18,TNA032
2,ChIJ-4q1aW-sQjQRIvwTiOkA-ho,雅雅愛犬專業用品店,雅雅愛犬專業美容,臺北市內湖區臺北市內湖區環山路一段63號1樓,114台灣臺北市內湖區環山路一段63號,0227986018,臺北市,內湖區,OPERATIONAL,"[星期一: 休息, 星期二: 10:00 – 19:00, 星期三: 10:00 – 19:...","[establishment, point_of_interest]",4.8,41.0,121.568218,25.086302,https://maps.google.com/?cid=1943867192174181410,None,2023-05-31,TPE010
3,ChIJ-5pqWYI9aTQRV4cgWYhIMJI,卡地亞犬坊,卡地亞犬舍,臺中市東區和平街259號1樓、地下1樓,401台灣臺中市東區和平街259号號,0933402020,臺中市,東區,OPERATIONAL,0,"[establishment, pet_store, point_of_interest, ...",3.7,3.0,120.689596,24.135867,https://maps.google.com/?cid=10533999278867646295,http://www.cartier-kennel.com/,2020-01-21,TCH002
4,ChIJ-6-t6kEFbjQRvxbTFcaRkaw,貓阿舍寵物旅館,貓阿舍精品貓旅館,高雄市三民區嫩江街129巷4號1、2樓,807台灣高雄市三民區嫩江街129巷4號,0928409122,高雄市,三民區,OPERATIONAL,"[星期一: 13:00 – 19:00, 星期二: 13:00 – 19:00, 星期三: ...","[establishment, point_of_interest]",5.0,6.0,120.298294,22.644752,https://maps.google.com/?cid=12434880326102357695,None,2025-06-09,KSH005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,ChIJzebFfxevQjQRJKrPXAQBrz8,林水梨商行,春松水果店紅樹林店/水果禮盒/現切水果/現打果汁/冰沙/竿蓁林輕軌站,新北市淡水區淡金路38巷33-20號1樓,251台灣新北市淡水區淡金路38巷35弄5號,0922232427,新北市,淡水區,OPERATIONAL,"[星期一: 休息, 星期二: 11:00 – 21:00, 星期三: 11:00 – 21:...","[establishment, food, grocery_or_supermarket, ...",4.2,10.0,121.457958,25.162027,https://maps.google.com/?cid=4588887663562435108,https://line.me/R/ti/g/VaI3n_jdku,2024-11-07,NTP010
1109,ChIJzwFWz5AZaDQR9h1DEVsCiL8,牠說寵物生活館,牠說寵物生活館/美容 住宿 安親,桃園市八德區桃園市八德區大和里金和路85號1樓,334台灣桃園市八德區金和路85號,033636320,桃園市,八德區,OPERATIONAL,"[星期一: 09:00 – 19:00, 星期二: 09:00 – 19:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.9,101.0,121.295934,24.954817,https://maps.google.com/?cid=13801283648231775734,https://www.facebook.com/WOUDLIKE/,2022-03-28,TYN008
1110,ChIJzwrXlEpTXTQRBY0aHtNV7cU,橘舍寵物沙龍旅館,AP寵物旅館x澡堂（寵物造型、寵物皮膚調理、寵物住宿、寵物零食、寵物保養品、寵物中途、寵物長照）,新北市汐止區秀峰路144號1樓,221台灣新北市汐止區吉祥街11巷8號,0924120216,新北市,汐止區,OPERATIONAL,0,"[establishment, point_of_interest]",4.8,48.0,121.666803,25.079955,https://maps.google.com/?cid=14262149960183745797,https://linkbio.co/6060214ZXCa10,2024-06-05,NTP011
1111,ChIJzxWfqJMEbjQRnbtlnXD8oQY,迷妳寵物店,迷妳寵物店,高雄市新興區復興一路14之3號,800台灣高雄市新興區復興一路14-3號,072225588,高雄市,新興區,OPERATIONAL,"[星期一: 12:00 – 22:00, 星期二: 12:00 – 22:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.2,31.0,120.308137,22.628589,https://maps.google.com/?cid=477940596085210013,https://www.facebook.com/%E8%BF%B7%E5%A6%B3%E5...,2024-10-09,KSH006


In [34]:
columns = [
    "key_0",
    "name_checked",
    "address_checked",
    "phone",
    "city",
    "district",
    "loc_id",
    "business_status",
    "opening_hours",
    "types",
    "rating",
    "rating_total",
    "longitude",
    "latitude",
    "map_url",
    "website",
    "newest_review",
]

df_final = df_final[columns]
df_final

,key_0,name_checked,address_checked,phone,city,district,loc_id,business_status,opening_hours,types,rating,rating_total,longitude,latitude,map_url,website,newest_review
0,ChIJ-1CF0w4daDQRRcdlKCG0yes,喵屋喵wu,220台灣新北市板橋區合宜一路27號,0922228719,新北市,板橋區,NTP001,OPERATIONAL,"[星期一: 11:00 – 19:30, 星期二: 11:00 – 19:30, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.9,36.0,121.443503,24.996627,https://maps.google.com/?cid=16990309123736586053,https://www.instagram.com/miuwumiuwu2020?igsh=...,2025-08-29
1,ChIJ-2CGR413bjQRtSti57eyD0A,小花朵寵物美容旅館《推薦寵物美容店》台南寵物皮膚調理｜住宿安親｜寄宿服務 ｜沐浴SPA｜專業...,701台灣臺南市東區小東路364號,0968985129,臺南市,東區,TNA032,OPERATIONAL,"[星期一: 休息, 星期二: 10:30 – 19:30, 星期三: 10:30 – 19:...","[establishment, pet_store, point_of_interest, ...",4.6,48.0,120.233784,22.998341,https://maps.google.com/?cid=4616104646008777653,https://www.facebook.com/ooxx9898new2.0,2024-01-18
2,ChIJ-4q1aW-sQjQRIvwTiOkA-ho,雅雅愛犬專業美容,114台灣臺北市內湖區環山路一段63號,0227986018,臺北市,內湖區,TPE010,OPERATIONAL,"[星期一: 休息, 星期二: 10:00 – 19:00, 星期三: 10:00 – 19:...","[establishment, point_of_interest]",4.8,41.0,121.568218,25.086302,https://maps.google.com/?cid=1943867192174181410,None,2023-05-31
3,ChIJ-5pqWYI9aTQRV4cgWYhIMJI,卡地亞犬舍,401台灣臺中市東區和平街259号號,0933402020,臺中市,東區,TCH002,OPERATIONAL,0,"[establishment, pet_store, point_of_interest, ...",3.7,3.0,120.689596,24.135867,https://maps.google.com/?cid=10533999278867646295,http://www.cartier-kennel.com/,2020-01-21
4,ChIJ-6-t6kEFbjQRvxbTFcaRkaw,貓阿舍精品貓旅館,807台灣高雄市三民區嫩江街129巷4號,0928409122,高雄市,三民區,KSH005,OPERATIONAL,"[星期一: 13:00 – 19:00, 星期二: 13:00 – 19:00, 星期三: ...","[establishment, point_of_interest]",5.0,6.0,120.298294,22.644752,https://maps.google.com/?cid=12434880326102357695,None,2025-06-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,ChIJzebFfxevQjQRJKrPXAQBrz8,春松水果店紅樹林店/水果禮盒/現切水果/現打果汁/冰沙/竿蓁林輕軌站,251台灣新北市淡水區淡金路38巷35弄5號,0922232427,新北市,淡水區,NTP010,OPERATIONAL,"[星期一: 休息, 星期二: 11:00 – 21:00, 星期三: 11:00 – 21:...","[establishment, food, grocery_or_supermarket, ...",4.2,10.0,121.457958,25.162027,https://maps.google.com/?cid=4588887663562435108,https://line.me/R/ti/g/VaI3n_jdku,2024-11-07
1109,ChIJzwFWz5AZaDQR9h1DEVsCiL8,牠說寵物生活館/美容 住宿 安親,334台灣桃園市八德區金和路85號,033636320,桃園市,八德區,TYN008,OPERATIONAL,"[星期一: 09:00 – 19:00, 星期二: 09:00 – 19:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.9,101.0,121.295934,24.954817,https://maps.google.com/?cid=13801283648231775734,https://www.facebook.com/WOUDLIKE/,2022-03-28
1110,ChIJzwrXlEpTXTQRBY0aHtNV7cU,AP寵物旅館x澡堂（寵物造型、寵物皮膚調理、寵物住宿、寵物零食、寵物保養品、寵物中途、寵物長照）,221台灣新北市汐止區吉祥街11巷8號,0924120216,新北市,汐止區,NTP011,OPERATIONAL,0,"[establishment, point_of_interest]",4.8,48.0,121.666803,25.079955,https://maps.google.com/?cid=14262149960183745797,https://linkbio.co/6060214ZXCa10,2024-06-05
1111,ChIJzxWfqJMEbjQRnbtlnXD8oQY,迷妳寵物店,800台灣高雄市新興區復興一路14-3號,072225588,高雄市,新興區,KSH006,OPERATIONAL,"[星期一: 12:00 – 22:00, 星期二: 12:00 – 22:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.2,31.0,120.308137,22.628589,https://maps.google.com/?cid=477940596085210013,https://www.facebook.com/%E8%BF%B7%E5%A6%B3%E5...,2024-10-09


In [38]:
df_final = df_final.rename(columns={"key_0": "place_id"})

In [44]:
df_final = df_final.sort_values(["city", "district"])
df_final

,place_id,name_checked,address_checked,phone,city,district,loc_id,business_status,opening_hours,types,rating,rating_total,longitude,latitude,map_url,website,newest_review
984,ChIJsZ4hTvMbaDQRWXEUzQDBHgk,巴寶寵物,237台灣新北市三峽區民生街2號,0226733589,新北市,三峽區,NTP009,OPERATIONAL,"[星期一: 休息, 星期二: 休息, 星期三: 10:30 – 18:30, 星期四: 10...","[establishment, point_of_interest]",4.8,22.0,121.376451,24.937043,https://maps.google.com/?cid=657174804827500889,None,2018-09-23
751,ChIJeza4mE4baDQR9LIJr9aE2DU,浪花寵藝寵物美容．用品．住宿,237台灣新北市三峽區長泰街48號一樓,0929387655,新北市,三峽區,NTP009,OPERATIONAL,"[星期一: 10:00 – 19:00, 星期二: 休息, 星期三: 休息, 星期四: 10...","[establishment, pet_store, point_of_interest, ...",4.9,64.0,121.378034,24.932523,https://maps.google.com/?cid=3879997136574198516,https://instagram.com/groomer.mei?igshid=MWI4M...,2023-08-14
479,ChIJQ90SF30baDQRDhhCkm5QzU0,合歡寵物旅館,23743台灣新北市三峽區溪東路233巷45弄28號,0987846016,新北市,三峽區,NTP009,OPERATIONAL,"[星期一: 10:00 – 18:00, 星期二: 10:00 – 18:00, 星期三: ...","[establishment, point_of_interest]",4.6,59.0,121.409502,24.934068,https://maps.google.com/?cid=5606225546976565262,http://www.dogshotel.tw/,2023-02-11
825,ChIJjTvRsfUbaDQRMSHrqLYJ6p0,台北寵物美容 寵物美容教學 台北寵物美容教學,237台灣新北市三峽區國學街1號,0956000075,新北市,三峽區,NTP009,OPERATIONAL,"[星期一: 10:00 – 19:00, 星期二: 10:00 – 19:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.2,124.0,121.373407,24.941599,https://maps.google.com/?cid=11378918088627659057,http://www.amopetgrooming.com/,2022-07-21
1092,ChIJyYK5LtK2QjQRSFxP28bkCX8,天麗寵物坊,252台灣新北市三芝區中山路一段88巷11號,0226369269,新北市,三芝區,NTP021,OPERATIONAL,"[星期一: 休息, 星期二: 09:00 – 20:00, 星期三: 09:00 – 20:...","[establishment, point_of_interest, store]",4.8,40.0,121.502673,25.256646,https://maps.google.com/?cid=9154099260341312584,https://www.facebook.com/TianLi26369269,2025-03-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,ChIJJ7QZqM0FbjQRhOoo8oiJtc8,沐寵物美容＆旅館《寵物沐浴推薦》鼓山專業寵物美容師｜寵物刷牙｜寵物安親 ｜韓國草本泥,804台灣高雄市鼓山區青松街25號,075310018,高雄市,鼓山區,KSH002,OPERATIONAL,"[星期一: 10:00 – 19:00, 星期二: 10:00 – 19:00, 星期三: ...","[establishment, point_of_interest]",5.0,50.0,120.279892,22.651296,https://maps.google.com/?cid=14967020157907036804,https://www.facebook.com/profile.php?id=100091...,2023-07-06
573,ChIJVzpG0lYEbjQR6ofvP6za-rg,優的生活寵物館/優的寵貓坊/推薦貓舍/高雄貓舍,804台灣高雄市鼓山區中華一路2220號,075503035,高雄市,鼓山區,KSH002,OPERATIONAL,"[星期一: 10:30 – 21:30, 星期二: 10:30 – 21:30, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.4,356.0,120.291783,22.653326,https://maps.google.com/?cid=13329206480498296810,https://www.facebook.com/petyono/,2025-06-26
476,ChIJQ-3NQNIFbjQReF1UhrUJmBs,貓膩Cafe,"2樓, No. 282號博愛一路三民區高雄市台灣 807",None,高雄市,鼓山區,KSH002,OPERATIONAL,"[星期一: 12:30 – 18:30, 星期二: 12:30 – 18:30, 星期三: ...","[cafe, establishment, food, point_of_interest,...",4.7,289.0,120.303660,22.648784,https://maps.google.com/?cid=1988349910731480440,https://www.instagram.com/maonicafe,2025-07-20
15,ChIJ-xWLMACnQjQRb-2Z-CaN1K8,星都企業有限公司,333台灣桃園市龜山區復興二路6號,None,高雄市,鼓山區,KSH002,OPERATIONAL,0,"[establishment, point_of_interest]",5.0,1.0,121.371255,25.053365,https://maps.google.com/?cid=12669906850219158895,None,2024-09-30


In [54]:
# 儲存ETL後CSV檔
path = "data/processed/pet_hotel_ETL.csv"
sd.store_to_csv_no_index(df_final, path)

[✓] 不含index的CSV檔已存檔完畢


In [46]:
# 新增hotel_id欄位
df_final["hotel_id"] = np.nan
df_final

,place_id,name_checked,address_checked,phone,city,district,loc_id,business_status,opening_hours,types,rating,rating_total,longitude,latitude,map_url,website,newest_review,hotel_id
984,ChIJsZ4hTvMbaDQRWXEUzQDBHgk,巴寶寵物,237台灣新北市三峽區民生街2號,0226733589,新北市,三峽區,NTP009,OPERATIONAL,"[星期一: 休息, 星期二: 休息, 星期三: 10:30 – 18:30, 星期四: 10...","[establishment, point_of_interest]",4.8,22.0,121.376451,24.937043,https://maps.google.com/?cid=657174804827500889,None,2018-09-23,NaN
751,ChIJeza4mE4baDQR9LIJr9aE2DU,浪花寵藝寵物美容．用品．住宿,237台灣新北市三峽區長泰街48號一樓,0929387655,新北市,三峽區,NTP009,OPERATIONAL,"[星期一: 10:00 – 19:00, 星期二: 休息, 星期三: 休息, 星期四: 10...","[establishment, pet_store, point_of_interest, ...",4.9,64.0,121.378034,24.932523,https://maps.google.com/?cid=3879997136574198516,https://instagram.com/groomer.mei?igshid=MWI4M...,2023-08-14,NaN
479,ChIJQ90SF30baDQRDhhCkm5QzU0,合歡寵物旅館,23743台灣新北市三峽區溪東路233巷45弄28號,0987846016,新北市,三峽區,NTP009,OPERATIONAL,"[星期一: 10:00 – 18:00, 星期二: 10:00 – 18:00, 星期三: ...","[establishment, point_of_interest]",4.6,59.0,121.409502,24.934068,https://maps.google.com/?cid=5606225546976565262,http://www.dogshotel.tw/,2023-02-11,NaN
825,ChIJjTvRsfUbaDQRMSHrqLYJ6p0,台北寵物美容 寵物美容教學 台北寵物美容教學,237台灣新北市三峽區國學街1號,0956000075,新北市,三峽區,NTP009,OPERATIONAL,"[星期一: 10:00 – 19:00, 星期二: 10:00 – 19:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.2,124.0,121.373407,24.941599,https://maps.google.com/?cid=11378918088627659057,http://www.amopetgrooming.com/,2022-07-21,NaN
1092,ChIJyYK5LtK2QjQRSFxP28bkCX8,天麗寵物坊,252台灣新北市三芝區中山路一段88巷11號,0226369269,新北市,三芝區,NTP021,OPERATIONAL,"[星期一: 休息, 星期二: 09:00 – 20:00, 星期三: 09:00 – 20:...","[establishment, point_of_interest, store]",4.8,40.0,121.502673,25.256646,https://maps.google.com/?cid=9154099260341312584,https://www.facebook.com/TianLi26369269,2025-03-09,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,ChIJJ7QZqM0FbjQRhOoo8oiJtc8,沐寵物美容＆旅館《寵物沐浴推薦》鼓山專業寵物美容師｜寵物刷牙｜寵物安親 ｜韓國草本泥,804台灣高雄市鼓山區青松街25號,075310018,高雄市,鼓山區,KSH002,OPERATIONAL,"[星期一: 10:00 – 19:00, 星期二: 10:00 – 19:00, 星期三: ...","[establishment, point_of_interest]",5.0,50.0,120.279892,22.651296,https://maps.google.com/?cid=14967020157907036804,https://www.facebook.com/profile.php?id=100091...,2023-07-06,NaN
573,ChIJVzpG0lYEbjQR6ofvP6za-rg,優的生活寵物館/優的寵貓坊/推薦貓舍/高雄貓舍,804台灣高雄市鼓山區中華一路2220號,075503035,高雄市,鼓山區,KSH002,OPERATIONAL,"[星期一: 10:30 – 21:30, 星期二: 10:30 – 21:30, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.4,356.0,120.291783,22.653326,https://maps.google.com/?cid=13329206480498296810,https://www.facebook.com/petyono/,2025-06-26,NaN
476,ChIJQ-3NQNIFbjQReF1UhrUJmBs,貓膩Cafe,"2樓, No. 282號博愛一路三民區高雄市台灣 807",None,高雄市,鼓山區,KSH002,OPERATIONAL,"[星期一: 12:30 – 18:30, 星期二: 12:30 – 18:30, 星期三: ...","[cafe, establishment, food, point_of_interest,...",4.7,289.0,120.303660,22.648784,https://maps.google.com/?cid=1988349910731480440,https://www.instagram.com/maonicafe,2025-07-20,NaN
15,ChIJ-xWLMACnQjQRb-2Z-CaN1K8,星都企業有限公司,333台灣桃園市龜山區復興二路6號,None,高雄市,鼓山區,KSH002,OPERATIONAL,0,"[establishment, point_of_interest]",5.0,1.0,121.371255,25.053365,https://maps.google.com/?cid=12669906850219158895,None,2024-09-30,NaN


In [50]:
num = df_final["hotel_id"].isna().sum()
num

np.int64(1113)

In [51]:
id_sign = "ht"
new_ids = [f"{id_sign}{str(i).zfill(4)}" for i in range(1, num + 1)]
df_final.loc[:, "hotel_id"] = new_ids
df_final

C:\Users\dinni\AppData\Local\Temp\ipykernel_22896\3732435375.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['ht0001', 'ht0002', 'ht0003', 'ht0004', 'ht0005', 'ht0006', 'ht0007', 'ht0008', 'ht0009', 'ht0010', 'ht0011', 'ht0012', 'ht0013', 'ht0014', 'ht0015', 'ht0016', 'ht0017', 'ht0018', 'ht0019', 'ht0020', 'ht0021', 'ht0022', 'ht0023', 'ht0024', 'ht0025', 'ht0026', 'ht0027', 'ht0028', 'ht0029', 'ht0030', 'ht0031', 'ht0032', 'ht0033', 'ht0034', 'ht0035', 'ht0036', 'ht0037', 'ht0038', 'ht0039', 'ht0040', 'ht0041', 'ht0042', 'ht0043', 'ht0044', 'ht0045', 'ht0046', 'ht0047', 'ht0048', 'ht0049', 'ht0050', 'ht0051', 'ht0052', 'ht0053', 'ht0054', 'ht0055', 'ht0056', 'ht0057', 'ht0058', 'ht0059', 'ht0060', 'ht0061', 'ht0062', 'ht0063', 'ht0064', 'ht0065', 'ht0066', 'ht0067', 'ht0068', 'ht0069', 'ht0070', 'ht0071', 'ht0072', 'ht0073', 'ht0074', 'ht0075', 'ht0076', 'ht0077', 'ht0078', 'ht0079', 'ht0080', 'ht0081', 'ht0

,place_id,name_checked,address_checked,phone,city,district,loc_id,business_status,opening_hours,types,rating,rating_total,longitude,latitude,map_url,website,newest_review,hotel_id
984,ChIJsZ4hTvMbaDQRWXEUzQDBHgk,巴寶寵物,237台灣新北市三峽區民生街2號,0226733589,新北市,三峽區,NTP009,OPERATIONAL,"[星期一: 休息, 星期二: 休息, 星期三: 10:30 – 18:30, 星期四: 10...","[establishment, point_of_interest]",4.8,22.0,121.376451,24.937043,https://maps.google.com/?cid=657174804827500889,None,2018-09-23,ht0001
751,ChIJeza4mE4baDQR9LIJr9aE2DU,浪花寵藝寵物美容．用品．住宿,237台灣新北市三峽區長泰街48號一樓,0929387655,新北市,三峽區,NTP009,OPERATIONAL,"[星期一: 10:00 – 19:00, 星期二: 休息, 星期三: 休息, 星期四: 10...","[establishment, pet_store, point_of_interest, ...",4.9,64.0,121.378034,24.932523,https://maps.google.com/?cid=3879997136574198516,https://instagram.com/groomer.mei?igshid=MWI4M...,2023-08-14,ht0002
479,ChIJQ90SF30baDQRDhhCkm5QzU0,合歡寵物旅館,23743台灣新北市三峽區溪東路233巷45弄28號,0987846016,新北市,三峽區,NTP009,OPERATIONAL,"[星期一: 10:00 – 18:00, 星期二: 10:00 – 18:00, 星期三: ...","[establishment, point_of_interest]",4.6,59.0,121.409502,24.934068,https://maps.google.com/?cid=5606225546976565262,http://www.dogshotel.tw/,2023-02-11,ht0003
825,ChIJjTvRsfUbaDQRMSHrqLYJ6p0,台北寵物美容 寵物美容教學 台北寵物美容教學,237台灣新北市三峽區國學街1號,0956000075,新北市,三峽區,NTP009,OPERATIONAL,"[星期一: 10:00 – 19:00, 星期二: 10:00 – 19:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.2,124.0,121.373407,24.941599,https://maps.google.com/?cid=11378918088627659057,http://www.amopetgrooming.com/,2022-07-21,ht0004
1092,ChIJyYK5LtK2QjQRSFxP28bkCX8,天麗寵物坊,252台灣新北市三芝區中山路一段88巷11號,0226369269,新北市,三芝區,NTP021,OPERATIONAL,"[星期一: 休息, 星期二: 09:00 – 20:00, 星期三: 09:00 – 20:...","[establishment, point_of_interest, store]",4.8,40.0,121.502673,25.256646,https://maps.google.com/?cid=9154099260341312584,https://www.facebook.com/TianLi26369269,2025-03-09,ht0005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,ChIJJ7QZqM0FbjQRhOoo8oiJtc8,沐寵物美容＆旅館《寵物沐浴推薦》鼓山專業寵物美容師｜寵物刷牙｜寵物安親 ｜韓國草本泥,804台灣高雄市鼓山區青松街25號,075310018,高雄市,鼓山區,KSH002,OPERATIONAL,"[星期一: 10:00 – 19:00, 星期二: 10:00 – 19:00, 星期三: ...","[establishment, point_of_interest]",5.0,50.0,120.279892,22.651296,https://maps.google.com/?cid=14967020157907036804,https://www.facebook.com/profile.php?id=100091...,2023-07-06,ht1109
573,ChIJVzpG0lYEbjQR6ofvP6za-rg,優的生活寵物館/優的寵貓坊/推薦貓舍/高雄貓舍,804台灣高雄市鼓山區中華一路2220號,075503035,高雄市,鼓山區,KSH002,OPERATIONAL,"[星期一: 10:30 – 21:30, 星期二: 10:30 – 21:30, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.4,356.0,120.291783,22.653326,https://maps.google.com/?cid=13329206480498296810,https://www.facebook.com/petyono/,2025-06-26,ht1110
476,ChIJQ-3NQNIFbjQReF1UhrUJmBs,貓膩Cafe,"2樓, No. 282號博愛一路三民區高雄市台灣 807",None,高雄市,鼓山區,KSH002,OPERATIONAL,"[星期一: 12:30 – 18:30, 星期二: 12:30 – 18:30, 星期三: ...","[cafe, establishment, food, point_of_interest,...",4.7,289.0,120.303660,22.648784,https://maps.google.com/?cid=1988349910731480440,https://www.instagram.com/maonicafe,2025-07-20,ht1111
15,ChIJ-xWLMACnQjQRb-2Z-CaN1K8,星都企業有限公司,333台灣桃園市龜山區復興二路6號,None,高雄市,鼓山區,KSH002,OPERATIONAL,0,"[establishment, point_of_interest]",5.0,1.0,121.371255,25.053365,https://maps.google.com/?cid=12669906850219158895,None,2024-09-30,ht1112


In [53]:
columns = [
    "hotel_id",
    "place_id",
    "name_checked",
    "address_checked",
    "phone",
    "city",
    "district",
    "loc_id",
    "business_status",
    "opening_hours",
    "types",
    "rating",
    "rating_total",
    "longitude",
    "latitude",
    "map_url",
    "website",
    "newest_review",
]
df_final = df_final[columns]
df_final

,hotel_id,place_id,name_checked,address_checked,phone,city,district,loc_id,business_status,opening_hours,types,rating,rating_total,longitude,latitude,map_url,website,newest_review
984,ht0001,ChIJsZ4hTvMbaDQRWXEUzQDBHgk,巴寶寵物,237台灣新北市三峽區民生街2號,0226733589,新北市,三峽區,NTP009,OPERATIONAL,"[星期一: 休息, 星期二: 休息, 星期三: 10:30 – 18:30, 星期四: 10...","[establishment, point_of_interest]",4.8,22.0,121.376451,24.937043,https://maps.google.com/?cid=657174804827500889,None,2018-09-23
751,ht0002,ChIJeza4mE4baDQR9LIJr9aE2DU,浪花寵藝寵物美容．用品．住宿,237台灣新北市三峽區長泰街48號一樓,0929387655,新北市,三峽區,NTP009,OPERATIONAL,"[星期一: 10:00 – 19:00, 星期二: 休息, 星期三: 休息, 星期四: 10...","[establishment, pet_store, point_of_interest, ...",4.9,64.0,121.378034,24.932523,https://maps.google.com/?cid=3879997136574198516,https://instagram.com/groomer.mei?igshid=MWI4M...,2023-08-14
479,ht0003,ChIJQ90SF30baDQRDhhCkm5QzU0,合歡寵物旅館,23743台灣新北市三峽區溪東路233巷45弄28號,0987846016,新北市,三峽區,NTP009,OPERATIONAL,"[星期一: 10:00 – 18:00, 星期二: 10:00 – 18:00, 星期三: ...","[establishment, point_of_interest]",4.6,59.0,121.409502,24.934068,https://maps.google.com/?cid=5606225546976565262,http://www.dogshotel.tw/,2023-02-11
825,ht0004,ChIJjTvRsfUbaDQRMSHrqLYJ6p0,台北寵物美容 寵物美容教學 台北寵物美容教學,237台灣新北市三峽區國學街1號,0956000075,新北市,三峽區,NTP009,OPERATIONAL,"[星期一: 10:00 – 19:00, 星期二: 10:00 – 19:00, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.2,124.0,121.373407,24.941599,https://maps.google.com/?cid=11378918088627659057,http://www.amopetgrooming.com/,2022-07-21
1092,ht0005,ChIJyYK5LtK2QjQRSFxP28bkCX8,天麗寵物坊,252台灣新北市三芝區中山路一段88巷11號,0226369269,新北市,三芝區,NTP021,OPERATIONAL,"[星期一: 休息, 星期二: 09:00 – 20:00, 星期三: 09:00 – 20:...","[establishment, point_of_interest, store]",4.8,40.0,121.502673,25.256646,https://maps.google.com/?cid=9154099260341312584,https://www.facebook.com/TianLi26369269,2025-03-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,ht1109,ChIJJ7QZqM0FbjQRhOoo8oiJtc8,沐寵物美容＆旅館《寵物沐浴推薦》鼓山專業寵物美容師｜寵物刷牙｜寵物安親 ｜韓國草本泥,804台灣高雄市鼓山區青松街25號,075310018,高雄市,鼓山區,KSH002,OPERATIONAL,"[星期一: 10:00 – 19:00, 星期二: 10:00 – 19:00, 星期三: ...","[establishment, point_of_interest]",5.0,50.0,120.279892,22.651296,https://maps.google.com/?cid=14967020157907036804,https://www.facebook.com/profile.php?id=100091...,2023-07-06
573,ht1110,ChIJVzpG0lYEbjQR6ofvP6za-rg,優的生活寵物館/優的寵貓坊/推薦貓舍/高雄貓舍,804台灣高雄市鼓山區中華一路2220號,075503035,高雄市,鼓山區,KSH002,OPERATIONAL,"[星期一: 10:30 – 21:30, 星期二: 10:30 – 21:30, 星期三: ...","[establishment, pet_store, point_of_interest, ...",4.4,356.0,120.291783,22.653326,https://maps.google.com/?cid=13329206480498296810,https://www.facebook.com/petyono/,2025-06-26
476,ht1111,ChIJQ-3NQNIFbjQReF1UhrUJmBs,貓膩Cafe,"2樓, No. 282號博愛一路三民區高雄市台灣 807",None,高雄市,鼓山區,KSH002,OPERATIONAL,"[星期一: 12:30 – 18:30, 星期二: 12:30 – 18:30, 星期三: ...","[cafe, establishment, food, point_of_interest,...",4.7,289.0,120.303660,22.648784,https://maps.google.com/?cid=1988349910731480440,https://www.instagram.com/maonicafe,2025-07-20
15,ht1112,ChIJ-xWLMACnQjQRb-2Z-CaN1K8,星都企業有限公司,333台灣桃園市龜山區復興二路6號,None,高雄市,鼓山區,KSH002,OPERATIONAL,0,"[establishment, point_of_interest]",5.0,1.0,121.371255,25.053365,https://maps.google.com/?cid=12669906850219158895,None,2024-09-30


In [3]:
# 儲存ETL後CSV檔
path = "data/processed/pet_hotel_ETL.csv"
sd.store_to_csv_no_index(df_final, path)

NameError: name 'df_final' is not defined

In [29]:
df = rd.get_csv_data("data/processed/pet_hotel_ETL.csv")
df

[✓] CSV檔案已取回



,hotel_id,place_id,name_checked,address_checked,phone,city,district,loc_id,business_status,opening_hours,types,rating,rating_total,longitude,latitude,map_url,website,newest_review
0,ht0001,ChIJsZ4hTvMbaDQRWXEUzQDBHgk,巴寶寵物,237台灣新北市三峽區民生街2號,226733589.0,新北市,三峽區,NTP009,OPERATIONAL,"['星期一: 休息', '星期二: 休息', '星期三: 10:30 – 18:30', '...","['establishment', 'point_of_interest']",4.8,22.0,121.376451,24.937043,https://maps.google.com/?cid=657174804827500889,NaN,2018-09-23
1,ht0002,ChIJeza4mE4baDQR9LIJr9aE2DU,浪花寵藝寵物美容．用品．住宿,237台灣新北市三峽區長泰街48號一樓,929387655.0,新北市,三峽區,NTP009,OPERATIONAL,"['星期一: 10:00 – 19:00', '星期二: 休息', '星期三: 休息', '...","['establishment', 'pet_store', 'point_of_inter...",4.9,64.0,121.378034,24.932523,https://maps.google.com/?cid=3879997136574198516,https://instagram.com/groomer.mei?igshid=MWI4M...,2023-08-14
2,ht0003,ChIJQ90SF30baDQRDhhCkm5QzU0,合歡寵物旅館,23743台灣新北市三峽區溪東路233巷45弄28號,987846016.0,新北市,三峽區,NTP009,OPERATIONAL,"['星期一: 10:00 – 18:00', '星期二: 10:00 – 18:00', '...","['establishment', 'point_of_interest']",4.6,59.0,121.409502,24.934068,https://maps.google.com/?cid=5606225546976565262,http://www.dogshotel.tw/,2023-02-11
3,ht0004,ChIJjTvRsfUbaDQRMSHrqLYJ6p0,台北寵物美容 寵物美容教學 台北寵物美容教學,237台灣新北市三峽區國學街1號,956000075.0,新北市,三峽區,NTP009,OPERATIONAL,"['星期一: 10:00 – 19:00', '星期二: 10:00 – 19:00', '...","['establishment', 'pet_store', 'point_of_inter...",4.2,124.0,121.373407,24.941599,https://maps.google.com/?cid=11378918088627659057,http://www.amopetgrooming.com/,2022-07-21
4,ht0005,ChIJyYK5LtK2QjQRSFxP28bkCX8,天麗寵物坊,252台灣新北市三芝區中山路一段88巷11號,226369269.0,新北市,三芝區,NTP021,OPERATIONAL,"['星期一: 休息', '星期二: 09:00 – 20:00', '星期三: 09:00 ...","['establishment', 'point_of_interest', 'store']",4.8,40.0,121.502673,25.256646,https://maps.google.com/?cid=9154099260341312584,https://www.facebook.com/TianLi26369269,2025-03-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,ht1109,ChIJJ7QZqM0FbjQRhOoo8oiJtc8,沐寵物美容＆旅館《寵物沐浴推薦》鼓山專業寵物美容師｜寵物刷牙｜寵物安親 ｜韓國草本泥,804台灣高雄市鼓山區青松街25號,75310018.0,高雄市,鼓山區,KSH002,OPERATIONAL,"['星期一: 10:00 – 19:00', '星期二: 10:00 – 19:00', '...","['establishment', 'point_of_interest']",5.0,50.0,120.279892,22.651296,https://maps.google.com/?cid=14967020157907036804,https://www.facebook.com/profile.php?id=100091...,2023-07-06
1109,ht1110,ChIJVzpG0lYEbjQR6ofvP6za-rg,優的生活寵物館/優的寵貓坊/推薦貓舍/高雄貓舍,804台灣高雄市鼓山區中華一路2220號,75503035.0,高雄市,鼓山區,KSH002,OPERATIONAL,"['星期一: 10:30 – 21:30', '星期二: 10:30 – 21:30', '...","['establishment', 'pet_store', 'point_of_inter...",4.4,356.0,120.291783,22.653326,https://maps.google.com/?cid=13329206480498296810,https://www.facebook.com/petyono/,2025-06-26
1110,ht1111,ChIJQ-3NQNIFbjQReF1UhrUJmBs,貓膩Cafe,"2樓, No. 282號博愛一路三民區高雄市台灣 807",NaN,高雄市,鼓山區,KSH002,OPERATIONAL,"['星期一: 12:30 – 18:30', '星期二: 12:30 – 18:30', '...","['cafe', 'establishment', 'food', 'point_of_in...",4.7,289.0,120.303660,22.648784,https://maps.google.com/?cid=1988349910731480440,https://www.instagram.com/maonicafe,2025-07-20
1111,ht1112,ChIJ-xWLMACnQjQRb-2Z-CaN1K8,星都企業有限公司,333台灣桃園市龜山區復興二路6號,NaN,高雄市,鼓山區,KSH002,OPERATIONAL,0,"['establishment', 'point_of_interest']",5.0,1.0,121.371255,25.053365,https://maps.google.com/?cid=12669906850219158895,NaN,2024-09-30


In [30]:
# 因為csv讀進來，list型態會變成string
# 將原本是字串的 list 轉回 Python list
def str_to_list(x):
    try:
        val = ast.literal_eval(x)

        # ✅ 如果轉完是 list 且裡面第一個元素也是 list
        # 表示外面多包了一層，要取第一層的內容
        if isinstance(val, list) and len(val) == 1 and isinstance(val[0], list):
            return val[0]

        # ✅ 正常 list
        if isinstance(val, list):
            return val

        # 不是 list，就包成 list
        return [val]

    except (ValueError, SyntaxError):
        return [x]

df["opening_hours"] = df["opening_hours"].apply(str_to_list)
df

,hotel_id,place_id,name_checked,address_checked,phone,city,district,loc_id,business_status,opening_hours,types,rating,rating_total,longitude,latitude,map_url,website,newest_review
0,ht0001,ChIJsZ4hTvMbaDQRWXEUzQDBHgk,巴寶寵物,237台灣新北市三峽區民生街2號,226733589.0,新北市,三峽區,NTP009,OPERATIONAL,"[星期一: 休息, 星期二: 休息, 星期三: 10:30 – 18:30, 星期四: 10...","['establishment', 'point_of_interest']",4.8,22.0,121.376451,24.937043,https://maps.google.com/?cid=657174804827500889,NaN,2018-09-23
1,ht0002,ChIJeza4mE4baDQR9LIJr9aE2DU,浪花寵藝寵物美容．用品．住宿,237台灣新北市三峽區長泰街48號一樓,929387655.0,新北市,三峽區,NTP009,OPERATIONAL,"[星期一: 10:00 – 19:00, 星期二: 休息, 星期三: 休息, 星期四: 10...","['establishment', 'pet_store', 'point_of_inter...",4.9,64.0,121.378034,24.932523,https://maps.google.com/?cid=3879997136574198516,https://instagram.com/groomer.mei?igshid=MWI4M...,2023-08-14
2,ht0003,ChIJQ90SF30baDQRDhhCkm5QzU0,合歡寵物旅館,23743台灣新北市三峽區溪東路233巷45弄28號,987846016.0,新北市,三峽區,NTP009,OPERATIONAL,"[星期一: 10:00 – 18:00, 星期二: 10:00 – 18:00, 星期三: ...","['establishment', 'point_of_interest']",4.6,59.0,121.409502,24.934068,https://maps.google.com/?cid=5606225546976565262,http://www.dogshotel.tw/,2023-02-11
3,ht0004,ChIJjTvRsfUbaDQRMSHrqLYJ6p0,台北寵物美容 寵物美容教學 台北寵物美容教學,237台灣新北市三峽區國學街1號,956000075.0,新北市,三峽區,NTP009,OPERATIONAL,"[星期一: 10:00 – 19:00, 星期二: 10:00 – 19:00, 星期三: ...","['establishment', 'pet_store', 'point_of_inter...",4.2,124.0,121.373407,24.941599,https://maps.google.com/?cid=11378918088627659057,http://www.amopetgrooming.com/,2022-07-21
4,ht0005,ChIJyYK5LtK2QjQRSFxP28bkCX8,天麗寵物坊,252台灣新北市三芝區中山路一段88巷11號,226369269.0,新北市,三芝區,NTP021,OPERATIONAL,"[星期一: 休息, 星期二: 09:00 – 20:00, 星期三: 09:00 – 20:...","['establishment', 'point_of_interest', 'store']",4.8,40.0,121.502673,25.256646,https://maps.google.com/?cid=9154099260341312584,https://www.facebook.com/TianLi26369269,2025-03-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,ht1109,ChIJJ7QZqM0FbjQRhOoo8oiJtc8,沐寵物美容＆旅館《寵物沐浴推薦》鼓山專業寵物美容師｜寵物刷牙｜寵物安親 ｜韓國草本泥,804台灣高雄市鼓山區青松街25號,75310018.0,高雄市,鼓山區,KSH002,OPERATIONAL,"[星期一: 10:00 – 19:00, 星期二: 10:00 – 19:00, 星期三: ...","['establishment', 'point_of_interest']",5.0,50.0,120.279892,22.651296,https://maps.google.com/?cid=14967020157907036804,https://www.facebook.com/profile.php?id=100091...,2023-07-06
1109,ht1110,ChIJVzpG0lYEbjQR6ofvP6za-rg,優的生活寵物館/優的寵貓坊/推薦貓舍/高雄貓舍,804台灣高雄市鼓山區中華一路2220號,75503035.0,高雄市,鼓山區,KSH002,OPERATIONAL,"[星期一: 10:30 – 21:30, 星期二: 10:30 – 21:30, 星期三: ...","['establishment', 'pet_store', 'point_of_inter...",4.4,356.0,120.291783,22.653326,https://maps.google.com/?cid=13329206480498296810,https://www.facebook.com/petyono/,2025-06-26
1110,ht1111,ChIJQ-3NQNIFbjQReF1UhrUJmBs,貓膩Cafe,"2樓, No. 282號博愛一路三民區高雄市台灣 807",NaN,高雄市,鼓山區,KSH002,OPERATIONAL,"[星期一: 12:30 – 18:30, 星期二: 12:30 – 18:30, 星期三: ...","['cafe', 'establishment', 'food', 'point_of_in...",4.7,289.0,120.303660,22.648784,https://maps.google.com/?cid=1988349910731480440,https://www.instagram.com/maonicafe,2025-07-20
1111,ht1112,ChIJ-xWLMACnQjQRb-2Z-CaN1K8,星都企業有限公司,333台灣桃園市龜山區復興二路6號,NaN,高雄市,鼓山區,KSH002,OPERATIONAL,[0],"['establishment', 'point_of_interest']",5.0,1.0,121.371255,25.053365,https://maps.google.com/?cid=12669906850219158895,NaN,2024-09-30


In [31]:
def trans_op_time_to_hours(op_time: list):
    """輸入營業時間list，會逐日計算營業時間並加總後回傳
    僅適用google map營業時間格式"""

    op_hours = 0

    # 營業時間為一週list，採逐日處理
    for day in op_time:
        if day != 0:
            # 使用分隔符號將"星期X"移除
            step1 = day.split(": ")[1].strip()

            # 若某日為休息則營業時間為0
            if step1 == "休息":
                hours = 0

            elif step1 == "24 小時營業":
                hours = 24

            else:
                # 若營業時間的字串中含有","表示不只一個營業時段
                # 先切割後再逐段處理
                if "," in day:
                    op_list = step1.split(",")
                    hours = 0
                    for period in op_list:
                        period = period.strip()
                        hours += dm.count_hours(period)
                else:
                    hours = dm.count_hours(step1)

            op_hours += hours

    return op_hours

In [32]:
df.loc[:, "opening_hours"] = df["opening_hours"].apply(trans_op_time_to_hours)
df

,hotel_id,place_id,name_checked,address_checked,phone,city,district,loc_id,business_status,opening_hours,types,rating,rating_total,longitude,latitude,map_url,website,newest_review
0,ht0001,ChIJsZ4hTvMbaDQRWXEUzQDBHgk,巴寶寵物,237台灣新北市三峽區民生街2號,226733589.0,新北市,三峽區,NTP009,OPERATIONAL,32.0,"['establishment', 'point_of_interest']",4.8,22.0,121.376451,24.937043,https://maps.google.com/?cid=657174804827500889,NaN,2018-09-23
1,ht0002,ChIJeza4mE4baDQR9LIJr9aE2DU,浪花寵藝寵物美容．用品．住宿,237台灣新北市三峽區長泰街48號一樓,929387655.0,新北市,三峽區,NTP009,OPERATIONAL,45.0,"['establishment', 'pet_store', 'point_of_inter...",4.9,64.0,121.378034,24.932523,https://maps.google.com/?cid=3879997136574198516,https://instagram.com/groomer.mei?igshid=MWI4M...,2023-08-14
2,ht0003,ChIJQ90SF30baDQRDhhCkm5QzU0,合歡寵物旅館,23743台灣新北市三峽區溪東路233巷45弄28號,987846016.0,新北市,三峽區,NTP009,OPERATIONAL,56.0,"['establishment', 'point_of_interest']",4.6,59.0,121.409502,24.934068,https://maps.google.com/?cid=5606225546976565262,http://www.dogshotel.tw/,2023-02-11
3,ht0004,ChIJjTvRsfUbaDQRMSHrqLYJ6p0,台北寵物美容 寵物美容教學 台北寵物美容教學,237台灣新北市三峽區國學街1號,956000075.0,新北市,三峽區,NTP009,OPERATIONAL,63.0,"['establishment', 'pet_store', 'point_of_inter...",4.2,124.0,121.373407,24.941599,https://maps.google.com/?cid=11378918088627659057,http://www.amopetgrooming.com/,2022-07-21
4,ht0005,ChIJyYK5LtK2QjQRSFxP28bkCX8,天麗寵物坊,252台灣新北市三芝區中山路一段88巷11號,226369269.0,新北市,三芝區,NTP021,OPERATIONAL,65.0,"['establishment', 'point_of_interest', 'store']",4.8,40.0,121.502673,25.256646,https://maps.google.com/?cid=9154099260341312584,https://www.facebook.com/TianLi26369269,2025-03-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108,ht1109,ChIJJ7QZqM0FbjQRhOoo8oiJtc8,沐寵物美容＆旅館《寵物沐浴推薦》鼓山專業寵物美容師｜寵物刷牙｜寵物安親 ｜韓國草本泥,804台灣高雄市鼓山區青松街25號,75310018.0,高雄市,鼓山區,KSH002,OPERATIONAL,54.0,"['establishment', 'point_of_interest']",5.0,50.0,120.279892,22.651296,https://maps.google.com/?cid=14967020157907036804,https://www.facebook.com/profile.php?id=100091...,2023-07-06
1109,ht1110,ChIJVzpG0lYEbjQR6ofvP6za-rg,優的生活寵物館/優的寵貓坊/推薦貓舍/高雄貓舍,804台灣高雄市鼓山區中華一路2220號,75503035.0,高雄市,鼓山區,KSH002,OPERATIONAL,77.0,"['establishment', 'pet_store', 'point_of_inter...",4.4,356.0,120.291783,22.653326,https://maps.google.com/?cid=13329206480498296810,https://www.facebook.com/petyono/,2025-06-26
1110,ht1111,ChIJQ-3NQNIFbjQReF1UhrUJmBs,貓膩Cafe,"2樓, No. 282號博愛一路三民區高雄市台灣 807",NaN,高雄市,鼓山區,KSH002,OPERATIONAL,44.5,"['cafe', 'establishment', 'food', 'point_of_in...",4.7,289.0,120.303660,22.648784,https://maps.google.com/?cid=1988349910731480440,https://www.instagram.com/maonicafe,2025-07-20
1111,ht1112,ChIJ-xWLMACnQjQRb-2Z-CaN1K8,星都企業有限公司,333台灣桃園市龜山區復興二路6號,NaN,高雄市,鼓山區,KSH002,OPERATIONAL,0.0,"['establishment', 'point_of_interest']",5.0,1.0,121.371255,25.053365,https://maps.google.com/?cid=12669906850219158895,NaN,2024-09-30


In [34]:
# 儲存ETL後CSV檔
path = "data/processed/pet_hotel_ETL.csv"
sd.store_to_csv_no_index(df, path)

[✓] 不含index的CSV檔已存檔完畢
